In [29]:
import numpy as np
import quant
from functools import reduce

In [25]:
def create_qft_matrix(n):
    """
    Generates Quantum Fourier Transformation matrix 
    
    Parameters
    ----------
    n: int
        Number of qubits to be transformed
    
    Return
    ------
    
    qft: array_like
        QFT matrix
    """
    N = 2 ** n
    w = croot(N, 1)
    norma = np.sqrt(N)
    res = np.empty([N, N], dtype=np.complex_)
    
    for x in range(N):
        
        temp = np.zeros([N, 1], dtype=np.complex_)
        
        for k in range(N):
            w_xk = w ** (x * k % N)
            vector_k = format(k, '0' + str(n) + 'b')
            column = quant.string_to_state(vector_k)
            temp += w_xk * column
            
        res[:, x] = temp[:, 0]
    
    return norma * res        

In [12]:
def croot(n, k):
    if n<=0:
        return None
    return np.exp((2 * np.pi * 1j * k) / n)
    

## Step 1 

In [68]:
N = 15

In [64]:
input_q = 4 
output_q = 4

In [65]:
x_0n = reduce(np.kron, np.repeat(quant.ZERO[np.newaxis, :], input_q, axis=0))
y_0n = reduce(np.kron, np.repeat(quant.ZERO[np.newaxis, :], output_q, axis=0))
input_vector = np.kron(x_0n, y_0n)

## Step 2
Hadamard stage

In [66]:
H = quant.hadamard_n(input_q)
I = reduce(np.kron, np.repeat(np.eye(2)[np.newaxis, :], output_q, axis=0))
HI = np.kron(H, I)

In [67]:
input_vector_s2 = np.dot(HI, input_vector)

## Step 3
Function 
$$ a^x mod N $$


In [101]:
def create_number_theory_matrix(n, func):
    N = 2 ** n
    res = np.empty([N, N])
    mid = n // 2 
    
    for num in range(N):
        state = format(num, '0' + str(n) + 'b')
        X = state[:mid]
        Y_ = int(state[mid:], 2) ^ func(int(X, 2))
        Y = format(Y_, '0' + str(mid) + 'b')
        column = quant.string_to_state(X + Y)
        
        res[:, num] = column[:, 0]
    
    return res  

In [46]:
def get_coprime(N):
    while True:
        a = randint(2, N-1)
        if gcd(a, N) == 1:
            return a

In [43]:
def gcd(a, b):
    while b != 0:
        tA = a % b
        a = b
        b = tA

    return a

In [76]:
def U_func(a, N):
    return lambda x : (a ** x) % N

In [109]:
a = get_coprime(N)
print(a)

7


In [110]:
U_f = create_number_theory_matrix(input_q + output_q, U_func(a, N))

In [111]:
input_vector_s3 = np.dot(U_f, input_vector_s2)

## Step 4

In [112]:
QFT = create_qft_matrix(input_q + output_q)

In [113]:
input_vector_s4 = np.dot(QFT, input_vector_s3)

## Step 5
Measuring

In [114]:
states = 2 ** input_q

for num in range(states):
    basis_str = format(num, '0' + str(input_q) + 'b')
    
    basis = quant.string_to_state(basis_str)
    prob = quant.measure(input_vector_s4, basis)
    print('|' + basis_str + '>', prob)

|0000> (6300.632673994301-1715.7480420251152j)
|0001> (-1303.9280996412372-1679.6061728420077j)
|0010> (-283.8783078839972+2075.6179138759853j)
|0011> (-1719.5181131738798-3688.198414172225j)
|0100> (-3055.4478584191943+616.6009373171348j)
|0101> (-2467.004119137943+2258.2191068166885j)
|0110> (2309.7675286752738+1194.3612643978418j)
|0111> (1755.3762955867076-1258.6674413585374j)
|1000> (3085.476773539388+1485.6593666807507j)
|1001> (1385.6283787551054-803.3145177525053j)
|1010> (604.079119443306-1348.1734501871492j)
|1011> (-5508.49242503301-1632.6852659142223j)
|1100> (-1550.4466638851336+2594.1667268667443j)
|1101> (-346.91108520529735-1387.547554602982j)
|1110> (-1266.1832654639234+1738.4434102941316j)
|1111> (2060.8491678495234+1550.8721326054292j)
